In [ ]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#subroutine to read in all experiments from single excel file
def readData(fname):
    xl = pd.ExcelFile(fname)
    sheet_names = xl.sheet_names #)  # see all sheet names
    #print(sheet_names)
    istart=0
    for sheet_name in sheet_names:
        if sheet_name=='Metadata':
            continue
        ds1 = xl.parse(sheet_name,header=0,skiprows=1)
        if istart==0:
            ds=ds1
            istart=1
        else:
            ds=pd.concat([ds,ds1]) #ds.append(ds1)
        #print(type(ds1))
        #print(type(ds))
        print(sheet_name,len(ds))
    return ds


In [ ]:
adir = 'C:/Users/gentemann/Google Drive/f_drive/docs/papers/in_prep/dave/'
file = adir + 'Duplicate_Master_Data_Jump_Test_2014_2017_2018.xlsx'
ds = readData(file)
df = pd.DataFrame(ds)


In [ ]:
#get some info about the data
#print out number of fish per experiment for table on data
print(ds.groupby('Experiment_number').fish_number.max())
cond_6 = (ds.height_difference<8)  #6" jumps
cond_12 = (ds.height_difference>8) & (ds.height_difference<15)  #12" jumps
cond_18 = (ds.height_difference>15)  #18" jumps
subset = ds[cond_6]
print('6',subset.groupby('Experiment_number').fish_number.max())
subset = ds[cond_12]
print('12',subset.groupby('Experiment_number').fish_number.max())
subset = ds[cond_18]
print('18',subset.groupby('Experiment_number').fish_number.max())


In [ ]:
# the histogram of the data
#fig = plt.figure()
#n, bins, patches = plt.hist(subset_lower.k_factor, 50, normed=1, facecolor='g', alpha=0.75)
#plt.xlabel('k_factor')
#plt.ylabel('Probability')
#plt.title('Histogram of k_factor')
#plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
#plt.axis([0, 2, 0, 0.03])
#plt.grid(True)
#fig.savefig(adir+'hist.png', dpi=300)

In [ ]:
#plot all fish
cond_lower = (ds.successful_jump==0)   # stayed in lower pool, unsuccessful jumper
cond_upper = (ds.successful_jump==1)  #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
fig = plt.figure()
h=plt.scatter(subset_lower.length,subset_lower.weight,color='r',marker='.',s=12,label="lower")
h=plt.scatter(subset_upper.length,subset_upper.weight,color='k',marker='.',s=12,label="upper")
plt.axis([0, 130,0, 15])
plt.ylabel('Weight (g)')
plt.xlabel('Length (mm)')
plt.legend()
fig.savefig(adir+'all_scatter.png', dpi=300)

In [ ]:
#6" jumps
#cond_lower = ((ds.successful_jump==0) & (ds.height_difference<10))  # stayed in lower pool, unsuccessful jumper
#cond_upper = ((ds.successful_jump==1) & (ds.height_difference<10)) #made it to upper pool
cond_lower = ((ds.successful_jump==0) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_6) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
fig = plt.figure()
h=plt.scatter(subset_lower.length,subset_lower.weight,color='r',marker='.',s=12,label="lower")
h=plt.scatter(subset_upper.length,subset_upper.weight,color='k',marker='.',s=12,label="upper")
plt.axis([0, 200,0, 70])
plt.ylabel('Weight (g)')
plt.xlabel('Length (mm)')
plt.legend()
fig.savefig(adir+'six_scatter.png', dpi=300)

In [ ]:
#12" jumps
cond_lower = ((ds.successful_jump==0) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_12) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
fig = plt.figure()
h=plt.scatter(subset_lower.length,subset_lower.weight,color='r',marker='.',s=12,label="lower")
h=plt.scatter(subset_upper.length,subset_upper.weight,color='k',marker='.',s=12,label="upper")
plt.axis([0, 200,0, 70])
plt.ylabel('Weight (g)')
plt.xlabel('Length (mm)')
plt.legend()
fig.savefig(adir+'twelve_scatter.png', dpi=300)

In [ ]:
#12" jumps
cond_lower = ((ds.successful_jump==0) & cond_18)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_18) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
fig = plt.figure()
h=plt.scatter(subset_lower.length,subset_lower.weight,color='r',marker='.',s=12,label="lower")
h=plt.scatter(subset_upper.length,subset_upper.weight,color='k',marker='.',s=12,label="upper")
plt.axis([0, 200,0, 70])
plt.ylabel('Weight (g)')
plt.xlabel('Length (mm)')
plt.legend()
fig.savefig(adir+'eighteen_scatter.png', dpi=300)
print(cond_lower.sum()+cond_upper.sum())
print(ds.length.max(),ds.weight.max())

In [ ]:
cols = df.columns.tolist()
print(cols)
cols_reorder = cols[12:13] + cols[15:16] + cols[14:15]
df_reorder=df[cols_reorder]
df_reorder.head()

In [ ]:
x = df_reorder.iloc[:,0:1].values  #just length  0.79 accuracy
#x = df_reorder.iloc[:,1:2].values    #just k-factor  0.71 accuracy
#x = df_reorder.iloc[:,0:2].values     #both length and k-factor but worried about their correlation  0.78 accuracy
y= df_reorder.iloc[:,2].values
# Split the data into Training and Testing set
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=0)
# Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)
#Fitting logistic regression to the training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(x_train,y_train)
# Predicting the Test set results
y_pred = classifier.predict(x_test)
# Making the confusion matrix 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
from sklearn import metrics
print (metrics.accuracy_score(y_test,classifier.predict(x_test)))

In [ ]:
cm

In [ ]:
#make figure showing prob of successful jump for different jump heights
fish_num_cutoff = 5
fish_length_int=ds.length.values.astype(int)
hbins = np.arange(0,200,1)-.5
cond_lower = ((ds.successful_jump==0) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_6) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_6 = hist_jump / tot_fish
prob_jump_6[cond_num]=np.nan
cond_lower = ((ds.successful_jump==0) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_12) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_12 = hist_jump / tot_fish
prob_jump_12[cond_num]=np.nan
cond_lower = ((ds.successful_jump==0) & cond_18)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_18) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_18 = hist_jump / tot_fish
prob_jump_18[cond_num]=np.nan
fig = plt.figure()
plt.plot(bin_edges[:-1],prob_jump_6, color='r')
plt.plot(bin_edges[:-1],prob_jump_12, color='b')
plt.plot(bin_edges[:-1],prob_jump_18, color='k')
plt.axis([0, 200,0, 1])
plt.ylabel('Probability Successful Jump (%)')
plt.xlabel('Length (mm)')
plt.text(10,.9,'6" jump',color='r')
plt.text(10,.84,'12" jump',color='b')
plt.text(10,.78,'18" jump',color='k')
fig.savefig(adir+'prob_1mmbins.png', dpi=300)
    

In [ ]:
#prob successful jump, in 5mm bins rather than 1mm bins
fish_num_cutoff = 10
fish_length_int=ds.length.values.astype(int)
hbins = np.arange(0,200,5)-.5
cond_lower = ((ds.successful_jump==0) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_6) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_6 = hist_jump / tot_fish
prob_jump_6[cond_num]=np.nan
cond_lower = ((ds.successful_jump==0) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_12) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_12 = hist_jump / tot_fish
prob_jump_12[cond_num]=np.nan
cond_lower = ((ds.successful_jump==0) & cond_18)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_18) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_18 = hist_jump / tot_fish
prob_jump_18[cond_num]=np.nan
fig = plt.figure()
plt.plot(bin_edges[:-1],prob_jump_6, color='r')
plt.plot(bin_edges[:-1],prob_jump_12, color='b')
plt.plot(bin_edges[:-1],prob_jump_18, color='k')
plt.axis([0, 200,0, 1])
plt.ylabel('Probability Successful Jump (%)')
plt.xlabel('Length (mm)')
plt.text(10,.9,'6" jump',color='r')
plt.text(10,.84,'12" jump',color='b')
plt.text(10,.78,'18" jump',color='k')
fig.savefig(adir+'prob_5mmbins.png', dpi=300)